# Imports

In [1]:
import random
import secrets
from decimal import *

# Functions

Returns the GCD of 2 numbers

In [2]:
def gcd(a, b):
    while(b):
        a, b = b, a % b 
    return a

Continued Fractions

In [3]:
from fractions import Fraction

def continued_fractions(convergence):
    
    convergence_list = []
    for i in range(1, len(convergence) + 1):
        convergence_list.append(convergence[:i])
    
    fractions = []
    
    for c in convergence_list:
        fractions.append(recursive_continued_fraction(c.copy()))
    
    return fractions
    
def recursive_continued_fraction(convergence):
    
    a = convergence.pop(0)
    
    if len(convergence) == 0:
        return a
    else:
        a = a + Fraction(1, (recursive_continued_fraction(convergence)))
        return a
    return a

In [4]:
def continued_fractions_expansion(a, b):
    expansion = []
    while(b):
        expansion.append(a // b)
        a, b = b, a % b 
    return expansion

Rabin-Miller Primality Test<br>
Checks if a number is prime, p is the number to be tested, s is the number of rounds<br>
Number of rounds should ensure that the probability of p not being prime is < 2^(-80) 

In [5]:
def RMPT(n, s):
    q = n - 1
    k = 0
    
    # Set n = 2^k*q, this while loop returns values k and q
    while q % 2 != 1:
        q //= 2
        k += 1
    
    # Number of iteration depends on security parameter S
    for _ in range(s):
        
        # Choose a witness
        a = random.randrange(2, n - 2)      
        
        # Set z = a^q mod n
        z = pow(a, q, n)
        
        # If z = 1 mod n or z = -1 mod n, then n is likely prime.
        if z == 1 or z == n - 1:
            continue #Not a witness, pick another a
        
        # Else, look through i=0 to k-1
        for i in range(1, k):
            # Square z: z = a^2 mod n
            z = pow(a, pow(2, i)*q, n)
            
            # If z = -1 mod n, then n is likely prime
            if z == n - 1:
                break #Not a witness, pick another a
                
        # After executing for loop, if a^((2^K)*Q)modP != -1 for all values of K, then a is a witness, therefore n is definitely not prime
        else:
            return False  
        
    return True #After S iterations still not a composite, chance to be a prime

Converts a string to ascii

In [6]:
def convert_to_ascii(message):
    ascii_message = ""
    for char in message:
        ascii_char = ord(char)
        ascii_char = str(ascii_char)
        ascii_char = ascii_char.zfill(3)
        ascii_message += ascii_char
    return int(ascii_message)

Converts ascii back to string

In [7]:
from textwrap import wrap

def convert_to_string(ascii_message):
    message = ""
    ascii_message = str(ascii_message)
    if(len(ascii_message) % 3 != 0):
        for _ in range(3 - len(ascii_message) % 3):
            ascii_message = "0" + ascii_message
    ascii_list = wrap(ascii_message, 3)
    for ascii in ascii_list:
        message = message + chr(int(ascii))
    return message

Generates the decryption key using public key e and 2 prime numbers p and q

In [8]:
def RSA_generate_D(e, p, q):
    N = p * q
    phi_N = (p - 1) * (q - 1)
    assert gcd(e, phi_N) == 1, "GCD of e and phi N must be 1"
    d = pow(e, -1, phi_N)
    return d

Encrypts a number with public key e

In [9]:
def RSA_encrypt(ascii_message, e, N):
    encrypted_message = pow(ascii_message, e, N)
    return encrypted_message

Encrypts a number using private key d

In [10]:
def RSA_decrypt(encrypted_message, d, N):
    ascii_message = pow(encrypted_message, d, N)
    return ascii_message

Continuously generates random bit_length numbers<br>
If number is even, adds one to make it odd<br>
Tries to factorize the number with small primes to determine if composite<br>
Uses Rabin-Miller Primality Test if it is still possible to be a prime number

In [11]:
def Generate_Prime(bit_length, RMPT_rounds):   
    while(True):
        num = secrets.randbits(bit_length)
        
        #If number is even, make it odd
        if num%2 == 0:
            num += 1
        #Try factorize using small primes
        elif num%3 == 0:
            continue
        elif num%5 == 0:
            continue
        elif num%7 == 0:
            continue
        elif num%11 == 0:
            continue
        #Try the expensive Rabin-Miller Primality Test
        elif RMPT(num, RMPT_rounds):
            print(num)
            return num

# RSA 1024

Generate 2 random 1024 bit primes

In [12]:
p = Generate_Prime(1024, 3)

80898932609390239944232389542832668529884078467540367835365016600412364131653227647497642289227017841921384330418676971617032705723603096904381778807713583739687930761964288006840764934629584700659068438946293238989367433990626115990645439321715363045696197433696039622303117088690197266492390210073898044313


In [13]:
q = Generate_Prime(1024, 3)

2630271862505791785486797041386366840316369562888191264489607498483342945736434028104215174145943207817664831851705948332226554498391767262873399791981078948946684267023356055768351204588749700721014672323623950787088713634147267747830491305271931535206549045775050339535393088630554884499773888866673703611


In [14]:
N = p * q

If we are designing a program that has a high emphasis on performance, in particular having high performance decryption, using a large decryption exponent would mean it would take longer.

Lets say that instead of using e to encrypt your message, you instead choose to use a much larger d to encrypt, using the smaller e to decrypt the message.

This should increase the performance!

But doing so would allow someone to use Wiener's attack to retrieve your decryption exponent since the decryption exponent is smaller than 1/3 * N ^ 1/4

# Wiener's attack

First lets randomly generate a small decryption exponent: Choose a 32 bit prime number

In [15]:
phi_N = (p-1) * (q-1)

# Generate a random 32 bit decryption exponent that satisfies GCD(d, phi_N) == 1
while(True):
    d = random.getrandbits(32)
    if gcd(d, phi_N) == 1:
        break;

In [16]:
print(d)

1068861653


Compute the encryption exponent: e = d^-1 mod phi(n)

In [17]:
e = RSA_generate_D(d, p, q)

In [18]:
print(e)

154582342869724303172654310050957233327513881024685873271089937275476121096683147295697736651249172279737505340006813372566853908256106087094822041959563410135337089304935776980453401762508004513863106306717642804680400402710117864712715715781397034921433636544450025641317316849084511138511667635248428497855825890109380101884160990742202087085928006574935362633485839050398654912079713367917330747625892519585618896638608502995905349612682575677839199599500262566878413186086109513489905951361745983779221066131457081659628420459721237945236054048838192707539313808945371676068470821409859684189667115186391529757


In [27]:
message = "Why using small decryption exponent is bad"

ascii_message = convert_to_ascii(message)
print(ascii_message)

87104121032117115105110103032115109097108108032100101099114121112116105111110032101120112111110101110116032105115032098097100


Encrypt message using e

In [28]:
encrypted_message = RSA_encrypt(ascii_message, e, N)

In [29]:
print(encrypted_message)

85646873753375238281338329956930320269852287169908439791438360911027329140988567157108005941558945091709834388004810264739672053709267505019846667001828571615306001695374768051550993392461939861488697964112323912558908348413451611100134969675136058248077789260773993354882663517927105414354436889608565528515496358084081189503509738731079570017649293531259869374501733813712472768838785448874680050599882489079173050763399751921502304549254015158727655900206649372261543312494095663724416335473852039467088894758516751843541415185061557349729978069302161797728508812280036172544128292137595109420274730032904946107


In [22]:
print(convert_to_string(encrypted_message))

Uʆͩ˱ŷîęŒŉμ΢ŀč͔ğ©ΌƷ̗ƶŨΏŉϜȷlέȮα[˅͂Ƅ̪Ĉˣʠ5˅ċǹ͎ʛ̼ȻɧĲʷŶ̀3ȦϡƈǍΫ͝ǨʹτpŃΐȮΌŜƝǃɣdωʣ:øM̕Ą̅ϡŢͲʗȅΟiƞŢƴ͹ɠȵȐȃǰŦTQ½Ƿǽˢ˛OȺʉĥȓăͥŶǵ˝̭ˈǘ̀͆̑ǀͪʨ2ɗͲǩO­2˻Ə˯ΙǶİȥþ˗ʏ΄ÎʉŴąȟĸǮ_ʗ˔ƠŏǙ͔'ǓX;˶Ȅ˯͋ȝƟ¹=ȭŝ˙ϒEĮ¡̝˘Ǽ̬Ę$¬ȠĤɓmƤĒ˚ Έβk


I can now calculate the convergents of e/N

In [30]:
expansion = continued_fractions_expansion(e, N)
fractions = continued_fractions(expansion)

One of the denominators will be d

In [24]:
denominators = []
for i in fractions:
    denominators.append(i.denominator)

(m^e)^d mod N

= m^1 mod N

= m mod N

We can encrypt a random message with e and test out all the denominators to see which one will result in getting the same message back

I.e. Test which demoninator d satisfies this equation: (M^e)^d = M

In [25]:
random_message = pow(17, e, N)

In [26]:
for denominator in denominators:
    if pow(random_message, denominator, N) == 17 :
        print(denominator)
        break;

1068861653


We found d!

# Conclusion

Make sure the decryption exponent is greater than 1/3 * N ^ 1/4 to prevent Wiener's attack, and greater than N^0.292 for Boneh and Durfee's improved Wiener's attack

But the best case is just not use a small decryption exponent in the first place